In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
document.title='AiiDAlab QE app'

<IPython.core.display.Javascript object>

In [2]:
# Fix pybel import path
try:
    import sys
    sys.modules['pybel'] = __import__('openbabel', globals(), locals(), ['pybel']).pybel
except Exception:
    pass

In [3]:
import urllib
import pprint
import requests

import aiida.orm
import optimade.adapters
import pymatgen.core.structure

from aiidalab_widgets_base.viewers import StructureDataViewer


class OptimadeInterceptor(StructureDataViewer):
    def __init__(self, url_key='optimade_structure', **kwargs):
        self._url_key = url_key
        structure = self._fetch_optimade_structure()
        self.aiida_structure = structure
        super().__init__(structure=structure, **kwargs)
        
    def _fetch_optimade_structure(self):
        # Get the URL from that passed to the notebook
        structure_url = urllib.parse.parse_qs(urllib.parse.urlparse(jupyter_notebook_url).query)[self._url_key][0]
        # Query the endpoint and get the aiida structure
        response = requests.get(structure_url)
        data = response.json()['data']
        structure = optimade.adapters.Structure(data).as_aiida_structuredata
        return structure

In [4]:
import ipywidgets as ipw
from jinja2 import Environment
from importlib_resources import files

from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import OptimadeQueryWidget, StructureExamplesWidget
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget, BasicStructureEditor
from aiidalab_widgets_base import WizardAppWidget, WizardAppWidgetStep
from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

from aiidalab_qe.app import static
from aiidalab_qe.common.process import QeAppWorkChainSelector
from aiidalab_qe.app.configuration import ConfigureQeAppWorkChainStep
from aiidalab_qe.app.submission import SubmitQeAppWorkChainStep
from aiidalab_qe.app.result import ViewQeAppWorkChainStatusAndResultsStep
from aiidalab_qe.app.structure import StructureSelectionStep, Examples
from aiidalab_qe.version import __version__


OptimadeQueryWidget.title = "OPTIMADE"  # monkeypatch


# Create the application steps
structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureBrowserWidget(title="AiiDA database"),
        StructureExamplesWidget(title="From Examples", examples=Examples),
    ],
    editors=[
        BasicStructureEditor(title="Edit structure"),
    ],
    node_class='StructureData',
    storable=False,
)
viewer = OptimadeInterceptor()
configure_qe_app_work_chain_step = ConfigureQeAppWorkChainStep(auto_advance=True)
submit_qe_app_work_chain_step = SubmitQeAppWorkChainStep(auto_advance=True)
submit_qe_app_work_chain_step.input_structure = viewer.aiida_structure
view_qe_app_work_chain_status_and_results_step = ViewQeAppWorkChainStatusAndResultsStep()

# Link the application steps
# configure_qe_app_work_chain_step.previous_step_state = WizardAppWidgetStep.State.SUCCESS
ipw.dlink((configure_qe_app_work_chain_step, 'state'), (submit_qe_app_work_chain_step, 'previous_step_state'))
ipw.dlink((configure_qe_app_work_chain_step, 'configuration_parameters'), (submit_qe_app_work_chain_step, 'input_parameters'))

ipw.dlink((submit_qe_app_work_chain_step, 'process'), (view_qe_app_work_chain_status_and_results_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Configure work chain', configure_qe_app_work_chain_step),
        ('Choose computational resources', submit_qe_app_work_chain_step),
        ('Status & Results', view_qe_app_work_chain_status_and_results_step),
    ])

# Add process selection header
work_chain_selector = QeAppWorkChainSelector(layout=ipw.Layout(width='auto'))

def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0

        
work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((submit_qe_app_work_chain_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)

env = Environment()

template = files(static).joinpath("welcome.jinja").read_text()
style = files(static).joinpath("style.css").read_text()
welcome_message = ipw.HTML(env.from_string(template).render(style=style))
footer = ipw.HTML(f'<p style="text-align:right;">Copyright (c) 2022 AiiDAlab team (EPFL)&#8195Version: {__version__}</p>')

app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

output = ipw.Output()
install_create_github_issue_exception_handler(
    output,
    url='https://github.com/aiidalab/aiidalab-qe/issues/new',
    labels=('bug', 'automated-report'))

with output:
    display(welcome_message, app_with_work_chain_selector, footer)
    
configure_qe_app_work_chain_step.previous_step_state = WizardAppWidgetStep.State.SUCCESS

display(viewer)
display(output)

/opt/conda/lib/python3.7/site-packages/optimade/adapters/structures/aiida.py:73: ConversionWarning: No mass defined for <species(name='Ce')>, will default to setting mass to 1.0.
  ConversionWarning,
/opt/conda/lib/python3.7/site-packages/aiida/transports/transport.py:201: AiidaDeprecationWarning: method is deprecated, use `aiida.plugins.entry_point.get_entry_point_names` instead
  warnings.warn(message, AiidaDeprecationWarning)  # pylint: disable=no-member
/opt/conda/lib/python3.7/site-packages/aiida/schedulers/scheduler.py:93: AiidaDeprecationWarning: method is deprecated, use `aiida.plugins.entry_point.get_entry_point_names` instead
  warnings.warn(message, AiidaDeprecationWarning)  # pylint: disable=no-member
/opt/conda/lib/python3.7/site-packages/aiida/orm/computers.py:786: AiidaDeprecationWarning: this method is deprecated, use the `hostname` property instead
  warnings.warn('this method is deprecated, use the `hostname` property instead', AiidaDeprecationWarning)  # pylint: disa

OptimadeInterceptor(children=(HBox(children=(VBox(children=(NGLWidget(),), layout=Layout(width='60%')), Tab(ch…

Output()